In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlite3

In [31]:
#checking the tables in the sqlite_db

db = sqlite3.connect('hop_teaming.sqlite')
db.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""").fetchall()

[('hop',), ('npi_data',), ('taxonomy',), ('zipcode',)]

In [2]:
npi = pd.read_csv('npidata_pfile_20050523-20230212.csv', chunksize = 10000)

In [4]:
npi = next(npi)

C:\Users\tjsim\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3309: DtypeWarning: Columns (32,60,64,67,68,69,70,71,72,73,74,75,76,77,78,79,81,82,83,85,86,87,89,90,91,93,94,151,155,159,161,162,163,165,166,167,169,170,174,314) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


In [5]:
nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'iso-8859-1')

In [7]:
hop = pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)

In [9]:
hop = next(hop)

In [10]:
db = sqlite3.connect('hop_teaming.sqlite') #name sqlite then add dbs

for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 1000000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait']<50)] # Clean up the column names
    chunk.to_sql('hop', db, if_exists = 'append', index = False)             # Append the chunk to a calls table

0it [00:00, ?it/s]

In [11]:
db.close

<function Connection.close>

In [12]:
npi_columns = ['NPI', 
    'Entity Type Code',
    'Provider Organization Name (Legal Business Name)',
    'Provider Last Name (Legal Name)',
    'Provider First Name',
    'Provider Middle Name',
    'Provider Name Prefix Text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',
    'Healthcare Provider Taxonomy Code_1',
    'Healthcare Provider Taxonomy Code_2',
    'Healthcare Provider Taxonomy Code_3',
    'Healthcare Provider Taxonomy Code_4',
    'Healthcare Provider Taxonomy Code_5',
    'Healthcare Provider Taxonomy Code_6',
    'Healthcare Provider Taxonomy Code_7',
    'Healthcare Provider Taxonomy Code_8',
    'Healthcare Provider Taxonomy Code_9',
    'Healthcare Provider Taxonomy Code_10',
    'Healthcare Provider Taxonomy Code_11',
    'Healthcare Provider Taxonomy Code_12',
    'Healthcare Provider Taxonomy Code_13',
    'Healthcare Provider Taxonomy Code_14',
    'Healthcare Provider Taxonomy Code_15',
    'Healthcare Provider Primary Taxonomy Switch_1',
    'Healthcare Provider Primary Taxonomy Switch_2',
    'Healthcare Provider Primary Taxonomy Switch_3',
    'Healthcare Provider Primary Taxonomy Switch_4',
    'Healthcare Provider Primary Taxonomy Switch_5',
    'Healthcare Provider Primary Taxonomy Switch_6',
    'Healthcare Provider Primary Taxonomy Switch_7',
    'Healthcare Provider Primary Taxonomy Switch_8',
    'Healthcare Provider Primary Taxonomy Switch_9',
    'Healthcare Provider Primary Taxonomy Switch_10',
    'Healthcare Provider Primary Taxonomy Switch_11',
    'Healthcare Provider Primary Taxonomy Switch_12',
    'Healthcare Provider Primary Taxonomy Switch_13',
    'Healthcare Provider Primary Taxonomy Switch_14',
    'Healthcare Provider Primary Taxonomy Switch_15']

In [13]:
npi_keeper_columns = ['NPI', 
    'Entity Type Code',
    'Provider Organization Name (Legal Business Name)',
    'Provider Last Name (Legal Name)',
    'Provider First Name',
    'Provider Middle Name',
    'Provider Name Prefix Text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',
    'taxonomy_code']

In [14]:
db = sqlite3.connect('hop_teaming.sqlite')

for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
    list_df = [] 

    for i in range (1,16): #Searching for the taxonomy code and creating its own column
        yes = chunk[
                    chunk[f'Healthcare Provider Primary Taxonomy Switch_{i}'] == 'Y'
                ][['NPI', f'Healthcare Provider Taxonomy Code_{i}']
                 ].rename(columns={f'Healthcare Provider Taxonomy Code_{i}':'taxonomy_code'})

        list_df.append(yes) 
    
    final = pd.merge(left = chunk, right = pd.concat(list_df), how = 'left', on = 'NPI')[npi_keeper_columns] #Merging the taxonomy codes back to npi data 
    
    final.columns = [x.lower().replace(' ', '_') for x in final.columns] #cleaning column names
    final.to_sql('npi_data', db, if_exists = 'append', index = False) #building database



0it [00:00, ?it/s]

C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (71,74,75,78,79,82,83,86,87,90) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (32,67,70,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (32,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000

C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (75,78,79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data 

C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (32,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (32,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppD

C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (5,6,8,10,91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (5,6,7,10,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_15764\575694477.py:3: DtypeWarning: Columns (5,6,7,8,10,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data 

In [15]:
db.close()

In [16]:
nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'iso-8859-1')

nucc.columns = [x.lower().replace(' ', '_') for x in nucc.columns]

db = sqlite3.connect('hop_teaming.sqlite')

nucc.to_sql('taxonomy', db, if_exists = 'append', index = False)

In [17]:
db.close()

In [18]:
db = sqlite3.connect('hop_teaming.sqlite')

pd.read_excel('ZIP_CBSA_122021.xlsx').to_sql('zipcode', db, if_exists = 'append', index = False)

In [ ]:
# Created index for from_npi and to_npi

In [32]:
db = sqlite3.connect('hop_teaming.sqlite')

db.execute('CREATE INDEX from_npi ON hop(from_npi)')
db.execute('CREATE INDEX to_npi ON hop(to_npi)')

db.close()

In [ ]:
# query to create db

In [9]:
query = '''
WITH hospital_npi_data AS(
    SELECT 
        DISTINCT entity_type_code AS entity,
        npi,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        taxonomy_code
    FROM 
        npi_data
    WHERE
        entity_type_code = 2
)
    SELECT 
        to_npi,
        code,
        grouping,
        classification,
        specialization,
        zip,
        SUM(patient_count) AS total_patients 
    FROM 
        hop
    INNER JOIN hospital_npi_data 
    ON hop.to_npi = hospital_npi_data.npi
    LEFT JOIN taxonomy
    ON taxonomy_code = code
    LEFT JOIN zipcode
    ON zipcode.zip = hospital_npi_data.zipcode
    WHERE cbsa = 34980
    GROUP BY
        to_npi,
        entity,
        organization, 
        zipcode,
        taxonomy_code,
        code,
        grouping,
        classification,
        specialization 
    ORDER BY total_patients DESC
'''

In [10]:
with sqlite3.connect('hop_teaming.sqlite') as db: 
    referral_table = pd.read_sql(query,db)

In [11]:
referral_table  

,to_npi,code,grouping,classification,specialization,zip,total_patients
0,1003863580,207ZP0102X,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,37217,1733907
1,1124075635,291U00000X,Laboratories,Clinical Medical Laboratory,None,37217,1174953
2,1104202761,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,None,37232,1166496
3,1093741464,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,None,37072,1126139
4,1235186800,291U00000X,Laboratories,Clinical Medical Laboratory,None,37217,1075639
...,...,...,...,...,...,...,...
1625,1295982775,261QP2300X,Ambulatory Health Care Facilities,Clinic/Center,Primary Care,37072,12
1626,1013116169,111N00000X,Chiropractic Providers,Chiropractor,None,37067,11
1627,1225466782,261QP2000X,Ambulatory Health Care Facilities,Clinic/Center,Physical Therapy,37201,11
1628,1427011915,261QR0400X,Ambulatory Health Care Facilities,Clinic/Center,Rehabilitation,37129,11


In [26]:
referral_table.to_csv('referral_table.csv')